In [1]:
#! -*- coding:utf-8 -*-
import re, os, json, codecs, gc
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
import pandas as pd
from random import choice
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

from keras.layers import *
from keras.callbacks import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import adam_v2

In [2]:
train_df = pd.read_csv('training_set.csv')
test_df = pd.read_csv('first_test_set.csv')
submit_df = pd.read_csv('初赛-submit.csv')
print(train_df.shape)

(480024, 17)


In [116]:
train_df.columns

Index(['rec_id', 'create_time', 'address', 'event_desc', 'event_src_name',
       'main_type_name', 'sub_type_name', 'district_name', 'community_name',
       'cell_name', 'duty_grid_name', 'coordinate_x', 'coordinate_y',
       'event_level_name', 'report_patrol_name', 'dispose_region_name',
       'first_unit_name', 'text', 'label'],
      dtype='object')

In [3]:
cate_fea = ['address','event_desc','event_src_name','main_type_name','sub_type_name','district_name','community_name','duty_grid_name','event_level_name','report_patrol_name','dispose_region_name']
locate_fea = ['coordinate_x','coordinate_y']
# time_fea = ['delta_seconds']

In [4]:
train_df = train_df.fillna("空值")
test_df = test_df.fillna("空值")

train_df['text'] = train_df.loc[:,cate_fea[0]].copy()
test_df['text'] = test_df.loc[:,cate_fea[0]].copy()

for cate in cate_fea[1:]:
    train_df['text'] = train_df['text'].map(str) +' '+train_df[cate].map(str)
    test_df['text'] = test_df['text'].map(str) +' '+test_df[cate].map(str)

In [5]:
# Label Encode for label
lbl = LabelEncoder().fit(train_df['first_unit_name'])
train_df.loc[:,'label'] = lbl.transform(train_df['first_unit_name'])

In [6]:
#! -*- coding:utf-8 -*-
import re, os, json, codecs, gc
import numpy as np
import pandas as pd
from random import choice
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

from keras.layers import *
from keras.callbacks import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import adam_v2

maxlen = 128
config_path = "/home/ldp/competitions/bert/chinese_L-12_H-768_A-12/bert_config.json"
checkpoint_path = '/home/ldp/competitions/bert/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/home/ldp/competitions/bert/chinese_L-12_H-768_A-12/vocab.txt'

token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]')  
            else:
                R.append('[UNK]')  
        return R

tokenizer = OurTokenizer(token_dict)

def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

class data_generator:
    def __init__(self, data, batch_size=8, shuffle=True):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            
            if self.shuffle:
                np.random.shuffle(idxs)
            
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                y = d[1]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y[:, 0, :]
                    [X1, X2, Y] = [], [], []

from keras.metrics import top_k_categorical_accuracy
def acc_top2(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)
                    
def build_bert(nclass):
    bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

    for l in bert_model.layers[:]:
        l.trainable = True

    x1_in = Input(shape=(None,))
    x2_in = Input(shape=(None,))

    x = bert_model([x1_in, x2_in])
    x = Lambda(lambda x: x[:, 0])(x)
    p = Dense(nclass, activation='softmax')(x)


    model = Model([x1_in, x2_in], p)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=adam_v2.Adam(1e-5),
                  metrics=['accuracy', acc_top2])
    print(model.summary())
    return model

In [7]:
from tensorflow.keras.utils import to_categorical

DATA_LIST = []
for data_row in train_df.iloc[:].itertuples():
    DATA_LIST.append((data_row.text, to_categorical(data_row.label, len(lbl.classes_))))
DATA_LIST = np.array(DATA_LIST, dtype=object)

DATA_LIST_TEST = []
for data_row in test_df.iloc[:].itertuples():
    DATA_LIST_TEST.append((data_row.text, to_categorical(0, len(lbl.classes_))))
DATA_LIST_TEST = np.array(DATA_LIST_TEST,  dtype=object)

In [8]:
!mkdir bert_dump

mkdir: cannot create directory ‘bert_dump’: File exists


In [19]:
def run_cv(nfold, data, data_label, data_test):
    kf = KFold(n_splits=nfold, shuffle=True, random_state=520).split(data)
    train_model_pred = np.zeros((len(data), len(lbl.classes_)))
    test_model_pred = np.zeros((len(data_test), len(lbl.classes_)))

    for i, (train_fold, test_fold) in enumerate(kf):
        print('fold:',i)
        if i < 3:
            continue
        X_train, X_valid, = data[train_fold, :], data[test_fold, :]
        
        model = build_bert(len(lbl.classes_))
        early_stopping = EarlyStopping(monitor='val_acc', patience=3)
        plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=2)
        checkpoint = ModelCheckpoint('./bert_dump/' + str(i) + '_ckp.hdf5', monitor='val_acc', 
                                         verbose=2, save_best_only=False, mode='max',save_weights_only=True)
        
        train_D = data_generator(X_train, shuffle=True)
        valid_D = data_generator(X_valid, shuffle=True)
        test_D = data_generator(data_test, shuffle=False)
        
        model.fit(
            train_D.__iter__(),
            steps_per_epoch=len(train_D),
            epochs=3,
            validation_data=valid_D.__iter__(),
            validation_steps=len(valid_D),
            callbacks=[early_stopping, plateau, checkpoint],
        )
        
        # model.load_weights('./bert_dump/' + str(i) + '.hdf5')

        # return model
        train_model_pred[test_fold, :] =  model.predict(valid_D.__iter__(), steps=len(valid_D),verbose=1)
        test_model_pred += model.predict(test_D.__iter__(), steps=len(test_D),verbose=1)
        model.save_weights(f'bert_dump/{i}_bert.hdf5')
        del model; gc.collect()
        K.clear_session()
        
        # break
        
    return train_model_pred, test_model_pred

In [20]:
train_model_pred, test_model_pred = run_cv(4, DATA_LIST, None, DATA_LIST_TEST)

fold: 0
fold: 1
fold: 2
fold: 3
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 model_1 (Functional)           (None, None, 768)    101677056   ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 768)          0      

In [21]:
submit_df['first_unit_name'] = lbl.inverse_transform(np.argmax(test_model_pred, 1))

In [ ]:
submit_df.to_csv('submit.csv',index = False)

In [ ]:
# train_model_pred.to_csv('bert_train_result.csv',index = False)